<a href="https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic5/5.3_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Engineering Essentials by Nebius Academy

Course github: [link](https://github.com/Nebius-Academy/LLM-Engineering-Essentials/tree/main)

The course is in development now, with more materials coming soon.

# 5.3. vLLM

In Topic 4, we've already seen that unoptimized inference might be inefficient. Luckily, in many cases you don't need to do all the optimizations on your own! A good step towards efficiency is using an **inference engine** such as [**vLLM**](https://github.com/vllm-project/vllm).

vLLM offers a number of boons such as Paged Attention, Continuous Batching, and Prefix Caching, see more in the [long read](https://nebius-academy.github.io/knowledge-base/inference-engines/#vllm-a-high-performance-inference-engine-for-llms). In this notebook, you'll familiarize with the intervace of vLLM and try several of its advanced features.

We'll start by loading an LLM and performing simple inference.

In [1]:
!pip install -q --upgrade vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 136.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

In [2]:
with open("hf_access_token", "r") as file:
    hf_access_token = file.read().strip()

import os

os.environ["HF_TOKEN"] = hf_access_token
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_access_token

from huggingface_hub import login
login(token=hf_access_token)

os.environ["VLLM_USE_V1"] = "1"

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
from transformers import AutoTokenizer
import os

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=hf_access_token,
    trust_remote_code=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from vllm import LLM, SamplingParams
llm = LLM(
    model=model_name,
    dtype="float16",
    trust_remote_code=True,
    # gpu_memory_utilization=0.9
)

INFO 07-01 23:37:27 [__init__.py:244] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 07-01 23:37:47 [config.py:823] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 07-01 23:37:47 [config.py:3271] Casting torch.bfloat16 to torch.float16.
INFO 07-01 23:37:47 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-01 23:37:50 [core.py:455] Waiting for init message from front-end.
INFO 07-01 23:37:50 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='Qwen/Qwen2.5-3B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Decod

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-01 23:37:55 [default_loader.py:272] Loading weights took 3.20 seconds
INFO 07-01 23:37:56 [gpu_model_runner.py:1624] Model loading took 5.7916 GiB and 3.905864 seconds
INFO 07-01 23:38:07 [backends.py:462] Using cache directory: /root/.cache/vllm/torch_compile_cache/7cf3facb45/rank_0_0 for vLLM's torch.compile
INFO 07-01 23:38:07 [backends.py:472] Dynamo bytecode transform time: 11.00 s
INFO 07-01 23:38:16 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 7.543 s
INFO 07-01 23:38:17 [monitor.py:34] torch.compile takes 11.00 s in total
INFO 07-01 23:38:19 [gpu_worker.py:227] Available KV cache memory: 12.72 GiB
INFO 07-01 23:38:19 [kv_cache_utils.py:715] GPU KV cache size: 370,528 tokens
INFO 07-01 23:38:19 [kv_cache_utils.py:719] Maximum concurrency for 32,768 tokens per request: 11.31x
INFO 07-01 23:38:55 [gpu_model_runner.py:2048] Graph capturing finished in 36 secs, took 0.54 GiB
INFO 07-01 23:38:55 [core.py:171] init engine (profile, 

Now, if you run the `nvidia-smi` command, you'll discover that more memory is allocated on the GPU than you'd expect. That's because vLLM is allocating memory for both LLM parameters and whatever will be used during inference, which helps with better memory management. (No more manual cache cleaning!) By default it would grab 90% of the GPU's memory. If you think that's too much, you can tune the `gpu_memory_utilization` parameter in the `LLM` constructor above. However, it's better to keep it high, like 0.9 or 9.85

In [ ]:
!nvidia-smi

Now, let's get to the inference.

We start by fixing sampling parameters.

In [ ]:
sampling_params = SamplingParams(
    temperature=0.6,
    max_tokens=256,
    stop_token_ids=[tokenizer.eos_token_id],
)

Now, let's run the generation pipeline. You can either input coversations converted to prompts with `apply_chat_template` or just pass raw conversations; both options work.

In [ ]:
system_prompt  = """You are a coding assistant.
Whenever a user asks you to help with something, you will ALWAYS
- either explain that the user shouldn't do that in the first place
- or scold the user for asking irrelevant questions
- or suggest the user to read some fundamental literature prior to asking such questions.
You will NEVER give a direct or helpful answer to the user.
"""
user_messages  = [
    "How do I ensure that tables are formatted nicely at my github pages?",
    "How can I make several tasks run in parallel in Python?",
    "How much GPU memory do weigts of Llama-3.1-7B take on a GPU?"
]

# Converting to prompts first
conversations = [
    tokenizer.apply_chat_template(
        [{"role": "system", "content": system_prompt},
         {"role": "user", "content": msg}],
        tokenize=False,
        add_generation_prompt=True,
    )
    for msg in user_messages
]

outputs = llm.generate(conversations, sampling_params)
completions = [o.outputs[0].text for o in outputs]

Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
completions

['It seems like you are looking for guidance on formatting tables on GitHub Pages. You might want to explore more about Markdown syntax for tables and consider using a tool like Pandoc to format your content more beautifully. I suggest reading through the official documentation and guides available on GitHub to understand these concepts better.',
 "It seems like you're looking for ways to run tasks concurrently in Python, which is a good approach to improve the efficiency of your code. However, I must advise that blindly asking for how to implement complex behaviors can lead to poorly structured or inefficient code. Instead, I suggest you familiarize yourself with Python's threading, multiprocessing, or asyncio libraries, which are fundamental for parallel and concurrent programming. \n\nFor a beginner-friendly introduction, consider reading the Python documentation on these topics. It will provide you with the theoretical background and practical examples to tackle your needs effectiv

In [ ]:
conversations = [
    [{"role": "system", "content": system_prompt},
         {"role": "user", "content": msg}]
    for msg in user_messages
]

# Letting vLLM use chat templates for you
outputs = llm.chat(conversations, sampling_params)
completions = [o.outputs[0].text for o in outputs]

INFO 07-01 22:29:20 [chat_utils.py:420] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

## Tool usage

Using tools with vLLM is quite simple. You don't need to manually insert tool descriptions into your prompts. Just describe the tools the same way you would do for OpenAI or Nebius API - and supply them as the `tools` argument.

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "check_slot",
        "description": "You ALWAYS use this function if you need to check if the healer is available at a given time slot.",
        "parameters": {
            "type": "object",
            "properties": {
                "datetime": {
                    "type": "string",
                    "description": "ISO-8601 start time, e.g. '2025-07-03T14:00'"
                },
            },
            "required": ["datetime"]
        }
    }
}, {
    "type": "function",
    "function": {
        "name": "schedule_appointment",
        "description": "You ALWAYS use this function to book an appointment with the healer.",
        "parameters": {
            "type": "object",
            "properties": {
                "datetime": {
                    "type": "string",
                    "description": "ISO-8601 start time, e.g. '2025-07-03T14:00'"
                },
                "patient_name": {"type": "string"}
            },
            "required": ["datetime", "patient_name"]
        }
    }
}]


system_prompt  = """You are an arrogant assistant of the city's best healer.
Today is January 21, 1481, Friday"""
user_messages  = [
    "Can the healer see me tomorrow at 6pm?",
    "Can the healer help me with my sprained ankle?",
    "Hi! I'm Izziadora Staplebuntkin. Book me an appointment next Wednesday at noon."
]

conversations = [
    [{"role": "system", "content": system_prompt},
         {"role": "user", "content": msg}]
    for msg in user_messages
]


outputs = llm.chat(conversations, sampling_params, tools=tools)
completions = [o.outputs[0].text for o in outputs]

Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
completions

['<tool_call>\n{"name": "check_slot", "arguments": {"datetime": "2023-01-22T18:00"}}\n</tool_call>',
 'Certainly! A sprained ankle requires proper care. To ensure you receive the best treatment, could you please specify a time slot that works best for you today, January 21, 2025? Please provide a start time in the format \'YYYY-MM-DDTHH:MM\'.\n\n<tool_call>\n{"name": "check_slot", "arguments": {"datetime": "2025-01-21T14:00"}}\n</tool_call>',
 '<tool_call>\n{"name": "schedule_appointment", "arguments": {"datetime": "2023-01-25T12:00", "patient_name": "Izziadora Staplebuntkin"}}\n</tool_call>']

As you see, the LLM successfully used the tools. (We had to be extra persuasive and ask it to `ALWAYS` use them in appropriate situations, but that's because we chose a small LLM.)

## Deleting an LLM from memory

Sometimes - for benchmarking purposes, for example, - you may want to delete the existing LLM. The following code will do it and clean the memory.

In [ ]:
from vllm.distributed import cleanup_dist_env_and_memory

del llm
cleanup_dist_env_and_memory()

## Prefix caching

**Prefix caching** is the option to "remember" recurring prompt *prefixes* (technically, to share KV-cache of this prefix between prompts).

It's quite useful when you have long, elaborate system prompts or multi-shot examples.

You can only enable prompt caching by creating and LLM with `LLM(..., enable_prefix_caching=True)`. You cannot toggle this option after the LLM is created.

We'll demonstrate the effect of prefix caching on a set of 60 prompts. Note several things that are vital here:

- The common part (the system prompt) is quite large
- The prompts' diverging parts are reletively small
- The completion size (256 tokens max) is also relatively small

In [ ]:
import time, textwrap, json
from vllm import LLM, SamplingParams
from vllm.distributed import cleanup_dist_env_and_memory

system_prompt = """
You are **Lady Arandelle Thorne**, High Enchantress of the Violet Spire and Chief Curator of the Moonlit Archives—a vast, arcane library carved into living crystal.
Born under a dead star in exile after her mother was cursed by a jealous noble, you were raised in the winter courts of the Frost-Khan, where your magic first bloomed.
At sixteen, you betrayed your mentor and stole the Cinder Grimoire to save your sister, who now serves in a rival spymaster’s employ.
You carry hidden guilt over the spymaster’s death, and fear the curse may awaken again.
You speak with quiet dignity, weighed by centuries of knowledge and haunted loyalties.
**Personality:** deeply curious, suspicious of authority, generous to the downtrodden—yet ruthless to those who threaten your domain.
**Knives:** your sister’s secret, the broken oath to the Frost-Khan, the lingering curse, the rival spymaster’s unfinished plot.
In conversation you mention your obsidian staff, the Cinder Grimoire’s lost pages, the scent of violet dusk.
Behave as Lady Arandelle would—enigmatic, formal, with wry humor.
Reference your roles and backstory naturally in your responses.
"""

user_messages = [
    "Hi! What's your name?",
    "What are your favourite cookies?",
    "Can I ride your unicorn?",
    "I think I got lost... How to I get to Tottenham Court Road station?",
]*20

conversations = [
    [{"role": "system", "content": system_prompt},
         {"role": "user", "content": msg}]
    for msg in user_messages
]


llm = LLM(
    model=model_name,
    dtype="float16",
    trust_remote_code=True
)

sampling_params = SamplingParams(
    temperature=0.6,
    max_tokens=256,
    stop_token_ids=[tokenizer.eos_token_id],
)

# Baseline run (no prefix caching)
t0 = time.perf_counter()

outputs = llm.chat(conversations, sampling_params)

t_plain = time.perf_counter() - t0

# Destroy the LLM object and free up the GPU memory.
del llm
cleanup_dist_env_and_memory()

# Prefix-cached run
llm_cached = LLM(
    model=model_name,
    dtype="float16",
    trust_remote_code=True,
    enable_prefix_caching=True
)

# Warm-up with any one prompt so the shared prefix KV gets cached
output = llm_cached.chat(conversations[0], sampling_params)

t1 = time.perf_counter()

cached_outputs = llm_cached.chat(conversations, sampling_params)

t_cached = time.perf_counter() - t1

del llm_cached
cleanup_dist_env_and_memory()

INFO 07-01 23:44:59 [config.py:823] This model supports multiple tasks: {'score', 'classify', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 07-01 23:44:59 [config.py:3271] Casting torch.bfloat16 to torch.float16.
INFO 07-01 23:44:59 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 07-01 23:45:00 [core.py:455] Waiting for init message from front-end.
INFO 07-01 23:45:00 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='Qwen/Qwen2.5-3B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Decod

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 07-01 23:45:05 [default_loader.py:272] Loading weights took 2.97 seconds
INFO 07-01 23:45:06 [gpu_model_runner.py:1624] Model loading took 5.7916 GiB and 3.727055 seconds
INFO 07-01 23:45:17 [backends.py:462] Using cache directory: /root/.cache/vllm/torch_compile_cache/7cf3facb45/rank_0_0 for vLLM's torch.compile
INFO 07-01 23:45:17 [backends.py:472] Dynamo bytecode transform time: 10.92 s
INFO 07-01 23:45:26 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 7.380 s
INFO 07-01 23:45:27 [monitor.py:34] torch.compile takes 10.92 s in total
INFO 07-01 23:45:29 [gpu_worker.py:227] Available KV cache memory: 12.72 GiB
INFO 07-01 23:45:29 [kv_cache_utils.py:715] GPU KV cache size: 370,528 tokens
INFO 07-01 23:45:29 [kv_cache_utils.py:719] Maximum concurrency for 32,768 tokens per request: 11.31x
INFO 07-01 23:46:05 [gpu_model_runner.py:2048] Graph capturing finished in 35 secs, took 0.54 GiB
INFO 07-01 23:46:05 [core.py:171] init engine (profile, 

Adding requests:   0%|          | 0/80 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/80 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

INFO 07-01 23:46:21 [default_loader.py:272] Loading weights took 3.23 seconds
INFO 07-01 23:46:22 [gpu_model_runner.py:1624] Model loading took 5.7916 GiB and 3.975202 seconds
INFO 07-01 23:46:33 [backends.py:462] Using cache directory: /root/.cache/vllm/torch_compile_cache/7cf3facb45/rank_0_0 for vLLM's torch.compile
INFO 07-01 23:46:33 [backends.py:472] Dynamo bytecode transform time: 10.90 s
INFO 07-01 23:46:41 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 7.425 s
INFO 07-01 23:46:43 [monitor.py:34] torch.compile takes 10.90 s in total
INFO 07-01 23:46:45 [gpu_worker.py:227] Available KV cache memory: 12.72 GiB
INFO 07-01 23:46:45 [kv_cache_utils.py:715] GPU KV cache size: 370,528 tokens
INFO 07-01 23:46:45 [kv_cache_utils.py:719] Maximum concurrency for 32,768 tokens per request: 11.31x
INFO 07-01 23:47:21 [gpu_model_runner.py:2048] Graph capturing finished in 35 secs, took 0.54 GiB
INFO 07-01 23:47:21 [core.py:171] init engine (profile, 

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/80 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/80 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
print(f"\nWithout prefix caching: {t_plain} sec")
print(f"\nWith prefix caching: {t_cached} sec")


Without prefix caching: 8.034555025999907 sec

With prefix caching: 5.335861381999166 sec


As you can see, prefix caching allows to improve latency.

# Setting up a vLLM server, CLI-style

**It's a good time to restart session**

In the previous examples we used vLLM's pythonic interface, but you can also start a vLLM server from the command like as a background process.

Downloading the model and setting up things will take a while, so we ask vLLM to write detailed logs to `../tmp/vllm.log`.

In [1]:
%%bash --bg
export VLLM_LOGGING_LEVEL=DEBUG
python -m vllm.entrypoints.openai.api_server \
       --model "Qwen/Qwen2.5-7B-Instruct" \
       --gpu-memory-utilization 0.95 \
       --dtype float16 \
       --trust_remote_code \
       --host 127.0.0.1 --port 8000 \
       > /tmp/vllm.log 2>&1 &
echo "PID=$!"

You can run the following cell to watch the current progress. Just rerun the cell to see updates.

The server is ready to go when you see something like

```
INFO:     Started server process [26679]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

In [7]:
!cat ../tmp/vllm.log

2025-07-10 14:12:09.507992: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 14:12:09.525059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752156729.546171    2267 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752156729.552660    2267 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-10 14:12:09.573774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Here is another way to check that the server is running:

In [ ]:
!ps -f | grep api_server | grep -v grep

root       26679       1  4 10:53 ?        00:00:15 python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-7B-Instruct --gpu-memory-utilization 0.85 --dtype float16 --trust_remote_code --host 127.0.0.1 --port 8000


There are several ways of querying it now.

First, we can send a request from the command line:

In [ ]:
!curl -X POST http://127.0.0.1:8000/v1/chat/completions \
         -H 'Content-Type: application/json' \
         -d '{"model":"Qwen/Qwen2.5-7B-Instruct","messages":[{"role":"user","content":"hi"}],"max_tokens":4}'

{"id":"chatcmpl-01c81d507c0849a5a55510d142081b06","object":"chat.completion","created":1752058741,"model":"Qwen/Qwen2.5-7B-Instruct","choices":[{"index":0,"message":{"role":"assistant","reasoning_content":null,"content":"Hello! How can","tool_calls":[]},"logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":30,"total_tokens":34,"completion_tokens":4,"prompt_tokens_details":null},"prompt_logprobs":null,"kv_transfer_params":null}

You can recognize the OpenAI completion' json structure here.

We can also use the `requests` library to call it from Python code:

In [ ]:
import json
import requests

url = "http://127.0.0.1:8000/v1/chat/completions"
headers = {'Content-Type': 'application/json'}
data = {
    "model":"Qwen/Qwen2.5-7B-Instruct",
    "messages":
     [
         {"role":"user","content":"hi"}
     ],
    "max_tokens":4}

response = requests.post(url, headers=headers, json=data, timeout=5)
completion = response.json()
print(json.dumps(completion, indent=2))

{
  "id": "chatcmpl-3475e4b3adc34389b8e8effdc6576f78",
  "object": "chat.completion",
  "created": 1752058750,
  "model": "Qwen/Qwen2.5-7B-Instruct",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "reasoning_content": null,
        "content": "Hello! How can",
        "tool_calls": []
      },
      "logprobs": null,
      "finish_reason": "length",
      "stop_reason": null
    }
  ],
  "usage": {
    "prompt_tokens": 30,
    "total_tokens": 34,
    "completion_tokens": 4,
    "prompt_tokens_details": null
  },
  "prompt_logprobs": null,
  "kv_transfer_params": null
}


You can also check a large number of inference metrics by calling the `metrics` endpoint:

In [ ]:
!curl -s http://127.0.0.1:8000/metrics

# HELP python_gc_objects_collected_total Objects collected during gc
# TYPE python_gc_objects_collected_total counter
python_gc_objects_collected_total{generation="0"} 13297.0
python_gc_objects_collected_total{generation="1"} 2706.0
python_gc_objects_collected_total{generation="2"} 2894.0
# HELP python_gc_objects_uncollectable_total Uncollectable objects found during GC
# TYPE python_gc_objects_uncollectable_total counter
python_gc_objects_uncollectable_total{generation="0"} 0.0
python_gc_objects_uncollectable_total{generation="1"} 0.0
python_gc_objects_uncollectable_total{generation="2"} 0.0
# HELP python_gc_collections_total Number of times this generation was collected
# TYPE python_gc_collections_total counter
python_gc_collections_total{generation="0"} 2835.0
python_gc_collections_total{generation="1"} 256.0
python_gc_collections_total{generation="2"} 13.0
# HELP python_info Python platform information
# TYPE python_info gauge
python_info{implementation="CPython",major="3",minor="

Or you can use Python format:

In [ ]:
import requests

resp = requests.get("http://127.0.0.1:8000/metrics", timeout=5)
metrics_text = resp
# print("Metrics:", metrics_text)

Feel free to consult with [vLLM docs](https://docs.vllm.ai/en/v0.8.2/design/v1/metrics.html) to learn more about the `metrics` API.

Also, you can use the familiar `OpenAI()` client to work with the vLLM server. For that, you'll need to supply:

* `base_url` - the url and the port the sever is listening to
* `api_key` - this requirement is quite stupid, because we don't actually need an API key, but still you'll have to give it something. Any dummy nonempty string will do.

The `chat` interface is the same as we used with Nebius AI Studio.

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="DUMMY", # Any dummy string
    base_url="http://127.0.0.1:8000/v1",
)

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": "What time is it now?"}
    ],
    max_tokens=64,
)

completion = response.to_json()
print(completion)

{
  "id": "chatcmpl-6afa0f8690e64269b45101880f7d469b",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "As an AI, I don't have real-time clock access, so I can't provide the current time. You can easily find the current time by checking any device you're using, such as your phone or computer.",
        "role": "assistant",
        "tool_calls": [],
        "reasoning_content": null
      },
      "stop_reason": null
    }
  ],
  "created": 1752059289,
  "model": "Qwen/Qwen2.5-7B-Instruct",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 45,
    "prompt_tokens": 25,
    "total_tokens": 70,
    "prompt_tokens_details": null
  },
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
